### VIX based strategy

* VIX which stands for volatility index was introduced by JP Morgans & Co.
* It has been defined as nearly 'bullet=proof' indicator.
* The indiactor has proved 100% in last three decades except the time of recession.

We will try to re-=generate the strategy using python and its modules and then a do a time series analysis.

In [141]:
# importing libraries
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [76]:
vix_df = yf.download('^VIX')

[*********************100%***********************]  1 of 1 completed


In [77]:
vix_df['MA'] = vix_df['Close'].rolling(window=20).mean()
vix_df

Price,Close,High,Low,Open,Volume,MA
Ticker,^VIX,^VIX,^VIX,^VIX,^VIX,
Date,,,,,,
1990-01-02,17.240000,17.240000,17.240000,17.240000,0,NaN
1990-01-03,18.190001,18.190001,18.190001,18.190001,0,NaN
1990-01-04,19.219999,19.219999,19.219999,19.219999,0,NaN
1990-01-05,20.110001,20.110001,20.110001,20.110001,0,NaN
1990-01-08,20.260000,20.260000,20.260000,20.260000,0,NaN
...,...,...,...,...,...,...
2025-04-02,21.510000,23.660000,20.680000,22.299999,0,21.6865
2025-04-03,30.020000,30.020000,24.930000,26.379999,0,21.9440


In [71]:
vix_df['MA']

Date
1990-01-02   NaN
1990-01-03   NaN
1990-01-04   NaN
1990-01-05   NaN
1990-01-08   NaN
              ..
2025-04-02   NaN
2025-04-03   NaN
2025-04-04   NaN
2025-04-07   NaN
2025-04-08   NaN
Name: MA, Length: 8883, dtype: float64

In [68]:
# chekcing columns
vix_df.columns

MultiIndex([( 'Close', '^VIX'),
            (  'High', '^VIX'),
            (   'Low', '^VIX'),
            (  'Open', '^VIX'),
            ('Volume', '^VIX'),
            (    'MA',     '')],
           names=['Price', 'Ticker'])

Since the dataframe uses multindex for its columns, instead of standard columns like 'Close' or 'MA' means each column name is a tuple of two values, not a simple string.

### Buy signal triggers when the VIX rises 50% of its moving average, we will calulcate first month average.

We will now check if the Close value is 50% larger than Vix value of that day.

In [78]:
vix_df_clean = vix_df.dropna(subset=[('Close', '^VIX'), ('MA', '')])
vix_df_filt = vix_df_clean[vix_df_clean['Close', '^VIX'] > 1.5 * vix_df_clean['MA']]


In [79]:
vix_df_filt

Price,Close,High,Low,Open,Volume,MA
Ticker,^VIX,^VIX,^VIX,^VIX,^VIX,
Date,,,,,,
1990-08-06,35.910000,35.910000,35.910000,35.910000,0,20.5140
1990-08-07,32.750000,32.750000,32.750000,32.750000,0,21.3055
1994-04-04,23.870001,28.299999,23.610001,23.610001,0,15.4870
1997-10-30,38.200001,38.560001,35.840000,36.709999,0,23.4675
2001-09-17,41.759998,44.330002,39.770000,43.200001,0,24.9870
...,...,...,...,...,...,...
2024-12-18,27.620001,28.320000,14.820000,15.570000,0,14.9920
2024-12-19,24.090000,24.120001,20.160000,21.610001,0,15.3385


Now we have the data for rows where there is no NaN values. This helps in refining our strategy.

In [86]:
# Converting time series to datetime
series = (pd.Series(vix_df_filt.index).diff()) / (np.timedelta64(1,'D')) >= 30
# Also NaT stands for 'Not a time'

In [87]:
series[0] = True

In [90]:
# We will filter over our dataframe to get the dates where the condition is True
signals = vix_df_filt[series.values]

Let's do some quality checks

In [91]:
signals.shape

(22, 6)

In [93]:
sp_df = yf.download('^GSPC', start='1990-01-01')

[*********************100%***********************]  1 of 1 completed


In [94]:
sp_df

Price,Close,High,Low,Open,Volume
Ticker,^GSPC,^GSPC,^GSPC,^GSPC,^GSPC
Date,,,,,
1990-01-02,359.690002,359.690002,351.980011,353.399994,162070000
1990-01-03,358.760010,360.589996,357.890015,359.690002,192330000
1990-01-04,355.670013,358.760010,352.890015,358.760010,177000000
1990-01-05,352.200012,355.670013,351.350006,355.670013,158530000
1990-01-08,353.790009,354.239990,350.540009,352.200012,140110000
...,...,...,...,...,...
2025-04-01,5633.069824,5650.569824,5558.520020,5597.529785,4434500000
2025-04-02,5670.970215,5695.310059,5571.479980,5580.759766,4243830000


In [97]:
from pandas.tseries.offsets import DateOffset

In [96]:
test_ = sp_df[sp_df.index >= signals.index[0]]

In [98]:
signals.index[0] + DateOffset(month=6)

Timestamp('1990-06-06 00:00:00')

In [99]:
test_ = sp_df[(sp_df.index >= signals.index[0]) & (sp_df.index <= signals.index[0] + DateOffset(months=6))]

In [101]:
# Lets check again
test_

Price,Close,High,Low,Open,Volume
Ticker,^GSPC,^GSPC,^GSPC,^GSPC,^GSPC
Date,,,,,
1990-08-06,334.429993,344.859985,333.269989,344.859985,240400000
1990-08-07,334.829987,338.630005,332.220001,334.429993,231580000
1990-08-08,338.350006,339.209991,334.829987,334.829987,190400000
1990-08-09,339.940002,340.559998,337.559998,338.350006,155810000
1990-08-10,335.519989,339.899994,334.220001,339.899994,145340000
...,...,...,...,...,...
1991-01-31,343.929993,343.929993,340.470001,340.920013,204520000
1991-02-01,343.049988,344.899994,340.369995,343.910004,246670000


Now to calculate the changes we will take the Close column and accumulate the return.

In [104]:
# Calculating returns
(test_.Close.pct_change()+1 ).cumprod().dropna()

Ticker,^GSPC
Date,
1990-08-07,1.001196
1990-08-08,1.011721
1990-08-09,1.016476
1990-08-10,1.003259
1990-08-13,1.013187
...,...
1991-01-31,1.028407
1991-02-01,1.025775
1991-02-04,1.041593


In [108]:
returns = []

for i in range(len(signals)):
    subdf = sp_df[(sp_df.index >= signals.index[i]) & 
                  (sp_df.index <= signals.index[i] + DateOffset(months=6))]
    returns.append((subdf.Close.pct_change() +1).prod())

In [110]:
pd.Series(returns).mean()

C:\Users\aditya\AppData\Local\Temp\ipykernel_25000\2640432410.py:1: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  pd.Series(returns).mean()


1.061053517840161

In [113]:
pd.Series(returns) -1

0     Ticker
^GSPC    0.070687
dtype: float64
1     Ticker
^GSPC    0.035701
dtype: float64
2     Ticker
^GSPC    0.230247
dtype: float64
3     Ticker
^GSPC    0.122635
dtype: float64
4     Ticker
^GSPC    0.048426
dtype: float64
5     Ticker
^GSPC   -0.327113
dtype: float64
6     Ticker
^GSPC    0.086602
dtype: float64
7     Ticker
^GSPC    0.205903
dtype: float64
8     Ticker
^GSPC    0.116117
dtype: float64
9     Ticker
^GSPC    0.062629
dtype: float64
10    Ticker
^GSPC   -0.026947
dtype: float64
11    Ticker
^GSPC    0.074374
dtype: float64
12    Ticker
^GSPC    0.072259
dtype: float64
13    Ticker
^GSPC    0.036806
dtype: float64
14    Ticker
^GSPC    0.252754
dtype: float64
15     Ticker
^GSPC    0.17223
dtype: float64
16    Ticker
^GSPC    0.063669
dtype: float64
17    Ticker
^GSPC    0.173482
dtype: float64
18    Ticker
^GSPC   -0.116828
dtype: float64
19    Ticker
^GSPC    0.129797
dtype: float64
20    Ticker
^GSPC   -0.137924
dtype: float64
21    Ticker
^GSPC   -0.002331
dty

We should consider if the strategy is still profitable since in the past 3 triggers, there were mixed signals. 12%returns, -13% returns & -0.2% returns

In [116]:
pd.Series(returns)[:-3].mean()

C:\Users\aditya\AppData\Local\Temp\ipykernel_25000\4120301255.py:1: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  pd.Series(returns)[:-3].mean()


1.0712439556526503

If we take out last 3 entries and start off from last year, we would get 71% returns.

In [140]:
s = pd.Series(returns)
df1 = s.to_frame()